# Mixtures of conjugate priors 01

# _Josep Fortiana_ $\hskip6cm$ 2022-04-04

## The spinning coin

***
###### LaTeX macros
$\def\prob{P}$
$\def\argmax{\operatorname{arg\,max}}$
$\def\argmin{\operatorname{arg\,min}}$
$\def\borel{\operatorname{Borel}}$
$\def\cE{\cal E}$
$\def\cP{\cal P}$
$\def\R{\mathbb{R}}$ 
$\def\N{\mathbb{N}}$
$\def\Z{\mathbb{Z}}$
$\def\Ee{\operatorname{E}}$
$\def\va{\text{v.a.}}$
$\def\var{\operatorname{var}}$
$\def\cov{\operatorname{cov}}$
$\def\cor{\operatorname{cor}}$
$\def\binomdist{\operatorname{Binom}}$
$\def\berndist{\operatorname{Ber}}$
$\def\betabinomdist{\operatorname{Beta-Binom}}$
$\def\betadist{\operatorname{Beta}}$
$\def\expdist{\operatorname{Exp}}$
$\def\gammadist{\operatorname{Gamma}}$
$\def\hyperdist{\operatorname{Hypergeom}}$
$\def\hypergeomdist{\operatorname{Hypergeom}}$
$\DeclareMathOperator{\multinomialdist}{Multinomial}$
$\DeclareMathOperator{\multinomdist}{Multinom}$
$\def\poissondist{\operatorname{Poisson}}$
$\def\geomdist{\operatorname{Geom}}$
$\def\normaldist{\operatorname{N}}$
$\def\unifdist{\operatorname{Unif}}$
$\DeclareMathOperator{\indica}{\mathbb{1}}$
$\def\CondTo{\mathbin{|\mskip0.5mu}}$
***

<div><center><table class="image">
<caption align="top"><h1>Persi Diaconis</h1></caption>
<tr><td><img src="Persi.Diaconis.jpg" width="500"></td></tr>
</table></center></div>   

Stanford stat and math professor Persi Diaconis, formerly a professional magician, famously determined how many times a deck of cards  must be shuffled in order to give a mathematically random result (seven). He’s also dabbled in coin games. 

You might be interested in his 2013 video talk: [The Search for Randomness](https://www.youtube.com/watch?v=xit5LDwJVck)

The example below is taken from the paper by Persi Diaconis and Donald Ylvisaker (1985), _Quantifying prior opinion._
In: J.M. Bernardo et al (eds), _Bayesian Statistics 2,_ Elsevier Science Publishers B.V. (North-Holland) pp. 133-156.
A modified form of it appears in  Jim Albert (2009), Bayesian computation with R (2nd ed), Springer, pp. 49-52 (se Exercise below).

## The spinning coin

The problem investigated originates from the following fact: if a coin is spinned on its edge instead of being flipped, the proportion of heads or tails is not around 50% but rather such values as one third or two thirds are obtained.

<div><center><table class="image">
<tr><td><img src="spinning.coin.2.jpg" width="170"></td></tr>
</table></center></div>   

According to Diaconis, "the reasons for the bias are not hard to infer. The shape of the edge will be a strong determining factor - indeed, magicians have coins that are slightly shaved; the eye cannot detect the shaving, but the spun coin always comes up heads".

For $ n$ tosses of a spinning coin, the number $x$ of heads up is a $\operatorname{B}(n,\theta)$, and $\theta$'s prior pdf will typically be a bimodal function (i.e., presenting two local maxima). 

Hence it cannot be modelled with a $\operatorname{Beta}(\alpha,\beta)$, which has a single mode at:

$$
	\dfrac{\alpha-1}{\alpha+\beta-2}
$$

# Diaconis spinning coin - Computations

In [ ]:
require(MASS) # for truehist
#install.packages("extraDistr", dependencies=TRUE, repos= "https://cloud.r-project.org")
require(extraDistr) # needed for the Beta-binomial predictive prior

In [ ]:
N<-20000

## Theoretical prior pdf

### Mixture of two Beta distributions

In [ ]:
prior.a1<-10
prior.b1<-20
prior.a2<-20
prior.b2<-10
prior.gamma<-0.5

In [ ]:
options(repr.plot.width=10,repr.plot.height=5)
v<-seq(0,1,length=500)
old.par<-par(mfrow=c(1,2))
plot(v,dbeta(v,prior.a1,prior.b1),type="l",lwd=3.5,col="DarkGreen",xlab="x",ylab="Density",main="First pdf")
plot(v,dbeta(v,prior.a2,prior.b2),type="l",lwd=3.5,col="DarkGreen",xlab="x",ylab="Density",main="Second pdf")
par(old.par)

NB. The mixture of two r.v., $ X_{1}$, with pdf $f_{1}$, and $X_{2}$, with pdf $f_{2}$, and coefficients $(\lambda, 1-\lambda)$, $\lambda\in(0,1)$, is a r.v. with pdf $f=\lambda f_{1}+(1-\lambda) f_{2}$, which is NOT the same thing as the _sum_   

$$
    X=\lambda X_{1}+(1-\lambda) X_{2}$. 
$$

We are not concerned just now about the pdf of $X=\lambda X_{1}+(1-\lambda) X_{2}$.

If you want to know, is the _convolution_ of the pdf's of its summands, not its sum. 

In [ ]:
mixture.prior.pdf<-function(theta){
    return(prior.gamma*dbeta(theta,prior.a1,prior.b1)+(1-prior.gamma)*dbeta(theta,prior.a2,prior.b2))
    }

In [ ]:
options(repr.plot.width=5.5,repr.plot.height=5.5)
v<-seq(0,1,length=500)
plot(v,mixture.prior.pdf(v),type="l",lwd=3.5,col="DarkGreen",xlab="x",ylab="Density",main="Mixture prior")

In [ ]:
# Generating random numbers following the mixture prior
Ind<-rbinom(N,size=1,prob=prior.gamma)
Sim.prior.theta<-Ind*rbeta(N,shape1=prior.a1,shape2=prior.b1)+(1-Ind)*rbeta(N,shape1=prior.a2,shape2=prior.b2)

In [ ]:
options(repr.plot.width=5.5,repr.plot.height=5.5)
truehist(Sim.prior.theta,ylim=c(0,2.5),nbins=30,col="LightGreen",xlab="",ylab="",main="Mixture prior")
v<-seq(0,1,length=500)
lines(v,mixture.prior.pdf(v),lwd=2.5,col="DarkGreen")

## Simulated prior predictive pmf

In [ ]:
n<-10

In [ ]:
Sim.x.prior.pred<-rbinom(N,size=n,prob=Sim.prior.theta)

## Theoretical prior predictive pdf

Assume the prior pdf for $ \theta$ is:

$$
	h(\theta)=\gamma\cdot h_{1}(\theta)+(1-\gamma)\cdot h_{2}(\theta),
$$

and the likelihood is $\sim \binomdist(n,\theta)$: 

$$
        f(x\,|\,\theta)=\binom{n}{x}\, \theta^{x}\cdot(1-\theta)^{n-x},\mskip40mu x=0,1,\dots,n.
$$

We assume that both $h_{1}(\theta)$ and $h_{2}(\theta)$ belong to the conjugate family to the likelihood.

Then, the joint pdf is:

$$
	f(x,\theta)=\gamma\cdot f_{1}(x,\theta)+(1-\gamma)\cdot  f_{2}(x,\theta).
$$

where:

$$
	 f_{i}(x,\theta)=f(x\,|\,\theta)\cdot h_{i}(\theta),\quad i=1,2.
$$

Integrating out $\theta$, the marginal for $x$:

$$
	f(x)=\gamma\cdot f_{1}(x)+(1-\gamma)\cdot f_{2}(x),
$$	

where:

$$
	f_{i}(x)=\int f(x\,|\,\theta)\cdot h_{i}(\theta)\,d\theta, \quad i=1,2.
$$

Thus, the prior predictive pdf is a mixture, a convex combination of two pdf's, _partial prior predictive pdf's,_ with the same coefficients, $ \gamma$ and $1-\gamma$, as the prior. By virtue of the conjugacy property we assumed these partial prior predictive pdf's follow from the general formulas of the conjugate pair, so there is no need to explicitly integrate $\theta$ out. 

$$
\begin{align*}
	f_{1}\sim\operatorname{Beta-Binom}(n,\alpha_{1},\beta_{1})&=\operatorname{Beta-Binom}(10,10,20),\\
	f_{2}\sim\operatorname{Beta-Binom}(n,\alpha_{2},\beta_{2})&=\operatorname{Beta-Binom}(10,20,10),
\end{align*}
$$

In [ ]:
f1<-function(x){dbbinom(x,size=n,alpha=prior.a1,beta=prior.b1)}
f2<-function(x){dbbinom(x,size=n,alpha=prior.a2,beta=prior.b2)}

In [ ]:
Theor.prior.predictive.pmf<-function(x){
    return(prior.gamma*f1(x)+(1-prior.gamma)*f2(x))
    }

### Compare simulated and theoretical prior predictive pmf

In [ ]:
# Use tabulate() since table() does not register empty bins (resulting table has only positive frequencies), 
# First parameter in tabulate() must be a vector of positive integers
T.Sim.x.prior.pred<-tabulate(Sim.x.prior.pred+1)
f.Sim.x.prior.pred<-T.Sim.x.prior.pred/N
# To recover the list of integers from 0 to the maximum integer with nonnull frequency
u<-(1:length(T.Sim.x.prior.pred))-1

In [ ]:
ftu<-Theor.prior.predictive.pmf(u)
fsu<-f.Sim.x.prior.pred

In [ ]:
fstu<-cbind(fsu,ftu)
round(t(fstu),3)
options(repr.plot.width=6,repr.plot.height=7)
matplot(x=cbind(u-0.1,u+0.1),y=fstu,type="h",lwd=7, lty=1,col=c("LightGreen","DarkGreen"),
        ylim=c(0,0.17),xlab="x",ylab="Rel. Freqs./Probs.",
        main="Simulated and theoretical prior predictive pmf")
legend("topright",legend=c("Simulated prior pred. pmf","Theoretical prior pred. pmf"),
       lwd=8, lty=1,col=c("LightGreen","DarkGreen"))

## Simulated posterior sample of $ \theta$

In [ ]:
# Observed x
x.obs<-3

In [ ]:
x.obs.indexes<-(Sim.x.prior.pred==x.obs)
N.post<-sum(x.obs.indexes)     # Posterior sample size
Prop.N.post<-N.post/N  
N.post
round(Prop.N.post,2)

In [ ]:
Sim.post.theta<-Sim.prior.theta[x.obs.indexes]
str(Sim.post.theta)

## Theoretical posterior pdf 

From Bayes' formula:

$$
    h(\theta\,|\,x)=\dfrac{f(x,\theta)}{f(x)}
        =\dfrac{\gamma\cdot f_{1}(x,\theta)+(1-\gamma)\cdot  f_{2}(x,\theta)}
            {\gamma\cdot f_{1}(x)+(1-\gamma)\cdot f_{2}(x)}.
$$

We will use the obvious notation:

$$
    h_{i}(\theta\,|\,x)=\dfrac{f_{i}(x,\theta)}{f_{i}(x)}	\quad i=1,2.
$$

These functions are called _partial posterior pdf's._ 

Again, the conjugacy property allows us to derive them from the general formulas of the conjugate pair, that is, they belong to the same family as the priors with updated parameters.
The posterior pdf is:

$$
	h(\theta\,|\,x)=\widehat{\gamma}(x)\cdot h_{1}(\theta\,|\,x)
		+(1-\widehat{\gamma}(x))\cdot h_{2}(\theta\,|\,x),
$$

where the _posterior mixture weights_ are:

$$
	\widehat{\gamma}(x)=\dfrac{\gamma\cdot f_{1}(x)}{\gamma\cdot f_{1}(x)+(1-\gamma)\cdot f_{2}(x)}
$$

and $1-\widehat{\gamma}(x)$. Now, these weight are not the same as those in the prior and, furthermore, they depend on the observed data.

### Posterior weights

In [ ]:
# for the observed x
f1x<-f1(x.obs)
f2x<-f2(x.obs)
round(f1x,4)
round(f2x,4)
#[1] 0.2276
#[1] 0.0277

In [ ]:
posterior.gamma<-prior.gamma*f1x/(prior.gamma*f1x+(1-prior.gamma)*f2x)
round(posterior.gamma,4)
#[1] 0.8915

$$
    \widehat{\gamma}(x)=\dfrac{\gamma\cdot f_{1}(x)}{\gamma\cdot f_{1}(x)+(1-\gamma)\cdot f_{2}(x)}= 0.8915.
$$

### Partial posterior pdf's

$$
\begin{array}{l}
	h_{1}\sim\operatorname{Beta}(\alpha_{1}+x,\beta_{1}+n-x)=\operatorname{Beta}(10+3,20+10-3)
        =\operatorname{Beta}(13,27),\\[0.3cm]
	h_{2}\sim\operatorname{Beta}(\alpha_{2}+x,\beta_{2}+n-x)=\operatorname{Beta}(20+3,10+10-3)
        =\operatorname{Beta}(23,17).
\end{array}
$$

In [ ]:
posterior.a1<-prior.a1+x.obs
posterior.b1<-prior.b1+n-x.obs
posterior.a2<-prior.a2+x.obs
posterior.b2<-prior.b2+n-x.obs

In [ ]:
sprintf("Posterior parameters for the first component: (%2.0d,%2.0d)",posterior.a1,posterior.b1)
sprintf("Posterior parameters for the second component: (%2.0d,%2.0d)",posterior.a2,posterior.b2)

### Theoretical posterior pdf

In [ ]:
mixture.posterior.pdf<-function(theta){
    return(posterior.gamma*dbeta(theta,posterior.a1,posterior.b1)
    +(1-posterior.gamma)*dbeta(theta,posterior.a2,posterior.b2))}

$$
    0.8915\,\operatorname{Beta}(13,27)+0.1085\,\operatorname{Beta}(23,17).
$$

In [ ]:
# Plot the posterior pdf
options(repr.plot.width=6,repr.plot.height=6)
v<-seq(0,1,length=500)
plot(v,mixture.posterior.pdf(v),"l",lwd=2.5,col="DarkGreen",xlab="",ylab="",main="Mixture posterior")

### Compare simulated posterior sample and theoretical posterior pdf

In [ ]:
options(repr.plot.width=6,repr.plot.height=6)
truehist(Sim.post.theta,col="LightGreen")
lines(v,mixture.posterior.pdf(v),lwd=2.5,col="DarkGreen",xlab="",ylab="",main="Mixture posterior")

### Posterior expectation, var, sd

In [ ]:
Expected.post.theta<-mean(Sim.post.theta)
var.post.theta<-var(Sim.post.theta)
sd.post.theta<-sd(Sim.post.theta)
round(Expected.post.theta,3)
round(var.post.theta,3)
round(sd.post.theta,3)

## MAP estimator of $ \theta$

The posterior distribution attains its maximum value where $ \operatorname{Beta}(13,27)$ does:

$$
    \hat{\theta}_{MAP}=\dfrac{13-1}{13+27-2}=0.3158,
$$

which may be compared with the classical ML estimator:

$$
    \hat{\theta}_{ML}=\dfrac{x}{n}=0.4286,
$$

and the lower mode of the prior distribution:

$$
    m_{1}=\dfrac{10-1}{10+20-2}=0.3214.
$$

## Credible interval with a given probability

The easiest way to obtain an interval $ [a,b]$ of $\theta$ values such that its posterior probability is a given value, e.g., $\pi=0.95$ is to use a random sample from the posterior distribution.

Taking it quantile-symmetrical, i.e., such that both tails have probability $(1-\pi)/2=0.025$, the interval is:

In [ ]:
p<-0.95
q<-(1-p)/2
round(quantile(Sim.post.theta,p=c(q,p+q)),4)
#  2.5%  97.5% 
#0.1932 0.6400

## Exercise


Compute an HPD 95% credible interval for this problem